In [1]:
import pandas as pd
from pprint import pprint
from collections import defaultdict
import re

# Potential Problem:
- original star allele definition spreadsheet was generated from PharmVar according to RefSeq locus identification. I mistakenly identifed this as gene position.
- Need to go back in, remake the spreadsheet for both GRCh37 and GRCh38. One csv for each
- IF FEELING AMBITIOUS, expand to include genes not currently annotated by PharmVar
  - need to figure out how to map from NG_*** notation to both 37 and 38

# Things to do:
- Refactor code into functions instead of scripts (look into OOP?)
  - make the genome build callable from function
  - function should be dependent on:
    
    1) genome build csv
    
    2) input vcf
    
    3) make check to verify that vcf and genome build match
    
- Read in vcf file
  - scikit-allel -http://alimanfoo.github.io/2017/06/14/read-vcf.html
  - read_vcf()?
  - use vcfreader by Kamil Slowikowski
- Iterate through vcf and return keys to match into mtIDs
  - eventually return the star allele haplotypes

In [28]:
uri = '/Users/david.hedges/projects/pgx_tool/pgx/Star Allele Dictionary.xlsx' # Fix me!!
df = pd.read_excel(uri, na_filter=None)
#df

In [30]:
df = df.replace({'-':''}, regex=True)

changes = ['change_1','change_2','change_3','change_4','change_5',
           'change_6','change_7','change_8','change_9','change_10',
           'change_11','change_12','change_13','change_14','change_15',
           'change_16','change_17','change_18','change_19','change_20',
           'change_21','change_22','change_23','change_24','change_25',
           'change_26','change_27','change_28','change_29','change_30',
           'change_31','change_32','change_33','change_34','change_35',
           'change_36','change_37','change_38']

for i in changes:
    df[i] = df[i].str.replace(r'\(.*\)','')

# pop out mtID from dataframe
df = df.drop(['mtID'], axis=1)

In [31]:
# Start locations for genes
genes38 = {'CYP2A13':41088451,'CYP2C8':95036772,'CYP2C9':94938658,'CYP2C19':94762681,
    'CYP2D6':42126499,'CYP2F1':41114292,'CYP2J2':59893308,'CYP2R1':14878005,
    'CYP2S1':41193219,'CYP2W1':983155,'CYP3A7':99705036,'CYP3A43':99827626,
    'CYP4F2':15878023,'CYP26A1':93073475,'NUDT15':48037567}

genes37 = {'CYP2A13':41594356,'CYP2C8':96796529,'CYP2C9':96698350,'CYP2C19':96522463,
    'CYP2D6':42522501,'CYP2F1':41620312,'CYP2J2':60358980,'CYP2R1':14899551,
    'CYP2S1':41699115,'CYP2W1':1020100,'CYP3A7':99302660,'CYP3A43':99425636,
    'CYP4F2':15988834,'CYP26A1':94833232,'NUDT15':48611703}

chroms = {'CYP2A13':19,'CYP2C8':10,'CYP2C9':10,'CYP2C19':10,
    'CYP2D6':22,'CYP2F1':19,'CYP2J2':1,'CYP2R1':11,
    'CYP2S1':19,'CYP2W1':7,'CYP3A7':7,'CYP3A43':7,
    'CYP4F2':19,'CYP26A1':10,'NUDT15':13}

In [140]:
df['Gene'] = df['Allele'].map(lambda x: x.split('*',1)[0])
#df

## make change here, no need to split out the * when we just made a new column

In [141]:
# replace allele name with gene name
df_change['Allele'] = df['Allele'].map(lambda x: x.split('*',1)[0])
#df_change

In [143]:
# make list (set) of all variants within the gene
#variant_dict = df_change.set_index('Allele').T.to_dict('list')
variant_dict = df_change.T.to_dict('list')
#rint(variant_dict)

In [144]:
# remove empty strings and clean whitespace
for key, val in variant_dict.items():
    #key.strip()
    val[:] = [i.strip() for i in val if i != '']

#variant_dict

In [152]:
# create master list of variants for each gene
dd_variants = defaultdict(list)
for k,v in variant_dict.items():
    dd_variants[v[0]].append(v[1:])
    
for k,v in dd_variants.items():
    dd_variants[k] = set([i for j in v for i in j])
    
#dd_variants

# Maybe when the GRCh build problem is fixed, I can just make one mapping dict

In [154]:

mt_list = []
for i in dd_variants.keys():
    gene_start_38 = genes38[i]
    gene_name = i
    chrom_num = chroms[i]
    for j in dd_variants[i]:
        if '>' in j:
            var_type = 'substitution'
            pos = [int(x) for x in re.findall(r'\d+', j)][0]
            sub = j.replace(str(pos),'')
            geno_pos = gene_start_38 + pos
            ref = sub[0]
            alt = sub[-1]
            mt_list.append([chrom_num, gene_name, var_type, geno_pos, pos, ref, alt]) 
        if 'ins' in j:
            var_type = 'insertion'
            pos = [int(j) for j in re.findall(r'\d+', i)][0]
            geno_pos = gene_start_38 + pos
            ref = '-'
            alt = ''.join(x for x in j if not x.isdigit()).replace('_ins','')
            mt_list.append([chrom_num, gene_name, var_type, geno_pos, pos, ref, alt])
        if 'del' in j:
            var_type = 'deletion'
            pos = [int(j) for j in re.findall(r'\d+', i)][0]
            geno_pos = gene_start_38 + pos
            ref = ''.join(x for x in j if not x.isdigit()).replace('del','')
            alt = '-'
            mt_list.append([chrom_num, gene_name, var_type, geno_pos, pos, ref, alt]) 
mt_dict = {mt_list.index(i):i for i in mt_list}
#mt_dict

In [147]:
mt_lookup = []
for i in dd_variants.keys():
    gene_start_38 = genes38[i]
    gene_name = i
    chrom_num = chroms[i]
    for j in dd_variants[i]:
        if '>' in j:
            var_type = 'substitution'
            pos = [int(x) for x in re.findall(r'\d+', j)][0]
            sub = j.replace(str(pos),'')
            geno_pos = gene_start_38 + pos
            ref = sub[0]
            alt = sub[-1]
            mt_lookup.append((gene_name, var_type, pos, ref, alt)) 
        if 'ins' in j:
            var_type = 'insertion'
            pos = [int(j) for j in re.findall(r'\d+', j)][0]
            geno_pos = gene_start_38 + pos
            ref = '-'
            alt = ''.join(x for x in j if not x.isdigit()).replace('_ins','')
            mt_lookup.append((gene_name, var_type, pos, ref, alt))
        if 'del' in j:
            var_type = 'deletion'
            pos = [int(j) for j in re.findall(r'\d+', j)][0]
            geno_pos = gene_start_38 + pos
            ref = ''.join(x for x in j if not x.isdigit()).replace('del','')
            alt = '-'
            mt_lookup.append((gene_name, var_type, pos, ref, alt)) 
            
mt_lookup_dict_inverse = {mt_lookup.index(i):i for i in mt_lookup}
mt_lookup_dict = {y:x for x,y in mt_lookup_dict_inverse.items()}
#mt_lookup_dict

In [148]:
star_dict = df.set_index('Allele').T.to_dict('list')

# remove empty strings and clean whitespace
for key, val in star_dict.items():
    val[:] = [i.strip() for i in val if i != '']

In [133]:
profile_dict_inverse = defaultdict(list)
for k,v in star_dict.items():
    for i in v:
        if '>' in i:
            var_type = 'substitution'
            gene = v[-1]
            pos = [int(x) for x in re.findall(r'\d+', i)][0]
            sub = i.replace(str(pos),'')
            ref = sub[0]
            alt = sub[-1]
            active_variant = (gene,var_type,pos,ref,alt)
            profile_dict_inverse[k].append('mt'+str(mt_lookup_dict[active_variant]))
        
        if 'ins' in i:
            var_type = 'insertion'
            pos = [int(x) for x in re.findall(r'\d+', i)][0]
            gene = v[-1]
            ref = '-'
            alt = ''.join(x for x in i if not x.isdigit()).replace('_ins','')
            active_variant = (gene,var_type,pos,ref,alt)
            profile_dict_inverse[k].append('mt'+str(mt_lookup_dict[active_variant]))
        
        if 'del' in i:
            var_type = 'deletion'
            gene = v[-1]
            pos = [int(x) for x in re.findall(r'\d+', i)][0]
            ref = ''.join(x for x in i if not x.isdigit()).replace('del','')
            alt = '-'
            active_variant = (gene,var_type,pos,ref,alt)
            profile_dict_inverse[k].append('mt'+str(mt_lookup_dict[active_variant]))

In [149]:
#profile_dict_inverse

In [150]:
## Make mt_dict from vcf
